In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import os
import requests
import json
import math

import googleapiclient.discovery
import googleapiclient.errors

In [2]:
%run setAPIkey.py

In [3]:
API_KEY = os.getenv('YT_API')

In [4]:
api_service_name = 'youtube'
api_version = 'v3'
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=API_KEY)

In [5]:
channel_ID = 'UCr8O8l5cCX85Oem1d18EezQ' # Daniel Bourke's channel ID

channel_request = youtube.channels().list(
    part='snippet,contentDetails',
    id=channel_ID
)
channel_res = channel_request.execute()
#channel_res

In [6]:
uploadPlaylist_ID = channel_res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
#print('Upload Playlist ID: ' + uploadPlaylist_ID)

In [7]:
def playlist_requester(pageToken=None,uploadPlaylist_ID=uploadPlaylist_ID):
    playlist_request = youtube.playlistItems().list(
        part='snippet,contentDetails',
        maxResults=50,
        pageToken=pageToken,
        playlistId=uploadPlaylist_ID
    )
    playlist_res = playlist_request.execute()
    
    return playlist_res

playlist_res = playlist_requester()

In [8]:
TOTAL_UPLOADS = playlist_res['pageInfo']['totalResults']
nextPageToken = playlist_res['nextPageToken']
print(TOTAL_UPLOADS, nextPageToken)

listOfVideo_IDs = [ video_ID['contentDetails']['videoId'] for video_ID in playlist_res['items'] ] # first 50
print(len(listOfVideo_IDs))
#listOfVideo_IDs

251 CDIQAA
50


In [9]:
while TOTAL_UPLOADS > len(listOfVideo_IDs):
    
    # going through pages and extending list of video IDs
    
    nextpage_playlist_res = playlist_requester(nextPageToken)
    listOfVideo_IDs.extend([ video_ID['contentDetails']['videoId'] for video_ID in nextpage_playlist_res['items'] 
                            if video_ID['contentDetails']['videoId'] not in listOfVideo_IDs ]) # extend() for lists / #append only does one item at ta time / concating lists with + is very intenstive according to "Python for Data Analysis"
    if 'nextPageToken' in nextpage_playlist_res:
        nextPageToken = nextpage_playlist_res['nextPageToken']
    print('Number of Uploaded Videos: ' + str(len(listOfVideo_IDs)))
    #print(nextPageToken)

Number of Uploaded Videos: 100
Number of Uploaded Videos: 150
Number of Uploaded Videos: 200
Number of Uploaded Videos: 250
Number of Uploaded Videos: 251


In [12]:
listOfTitles = []
listOfTags = []
listOfViews = []
listOfLikes = []
listOfDislikes = []
listOfComments = []

for i in range(math.ceil(len(listOfVideo_IDs)/50)):
    video_request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=listOfVideo_IDs[50*i:50*(i+1)]
    )
    video_response = video_request.execute()

    for vid in video_response['items']:
        listOfTitles.append(vid['snippet']['title'])
        if 'tags' in vid['snippet']:
            listOfTags.append(vid['snippet']['tags'])
        else:
            listOfTags.append('NaN')
        listOfViews.append(vid['statistics']['viewCount'])
        listOfLikes.append(vid['statistics']['likeCount'])
        listOfDislikes.append(vid['statistics']['dislikeCount'])
        listOfComments.append(vid['statistics']['commentCount'])
 
    #print(len(video_response['items']))


In [20]:
#zip lists together
combinedList = list(zip(listOfTitles, listOfViews, listOfTags,
                  listOfLikes, listOfDislikes, listOfComments))

columns = ['Title', 'ViewCount', 'Tags', 'LikeCount', 'DislikeCount', 'CommentCount']

In [22]:
df = pd.DataFrame(combinedList, columns=columns)
df.head()

,Title,ViewCount,Tags,LikeCount,DislikeCount,CommentCount
0,The future will not be supervised... | Machine...,6832,"[self supervised learning machine learning, ma...",261,1,35
1,Are courses enough for a job? | Machine learni...,6774,NaN,296,8,25
2,What does a head of data science look for in a...,8985,"[ken jee, daniel bourke, what questions get as...",449,8,34
3,2020 Machine Learning Roadmap,216952,"[machine learning field guide, machine learnin...",11810,137,523
4,One Neural Network to Rule Them All (and more)...,5466,"[zero to mastery, machine learning monthly, wh...",198,0,62


In [23]:
df.tail()

,Title,ViewCount,Tags,LikeCount,DislikeCount,CommentCount
246,Text/Profanity Checker in Python,2763,"[udacity, coding, python, programming, ai, art...",19,1,10
247,Functional Friday | Backyard Workout,330,"[workout music 2016, workout motivation, worko...",5,0,0
248,High Protein Vegan Snack - Crunchy Spicy Chic...,353,"[cooking, recipes, vegetarian, snacks, protein...",11,1,0
249,HOW (not) TO BUILD A PULL UP BAR,456,"[DIY, fitness, exercise, pullupbar, snapchat, ...",9,0,5
250,Xbox One S Unboxing and Xbox One Comparison,248,"[xbox, xboxone, gaming, mlg, microsoft, xboxon...",5,0,3


In [24]:
df.shape

(251, 6)